#### Импорт необходимых библиотек

In [102]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from transformers import BertTokenizer, BertModel
import optuna
from gensim.models import Word2Vec
import librosa
import soundfile as sf
import random
from sklearn.metrics import f1_score, precision_score, recall_score
from scipy import signal

#### Константы

In [85]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    # 'hr_bot_clear',
    # 'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    # 'hr_bot_clear.json',
    # 'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

In [86]:
# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

Модель Vosk загружена успешно.


In [87]:
# Метки команд
_label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

#### Объявление классов

In [ ]:
# Функция для подавления шума в одном аудиофайле
def reduce_noise(audio_file, noise_file, output_file, noise_reduction_factor=1):
    voice, sr = librosa.load(audio_file, sr=16000)
    noise, sr_noise = librosa.load(noise_file, sr=16000)

    # Проверка на длину шума
    if len(noise) < len(voice):
        noise = np.pad(noise, (0, len(voice) - len(noise)), mode='wrap')
    else:
        noise = noise[:len(voice)]

    # Удаление шума
    voice_clean = nr.reduce_noise(y=voice, sr=sr, y_noise=noise, prop_decrease=noise_reduction_factor)

    # Сохранение очищенного файла
    sf.write(output_file, voice_clean, sr)
    return output_file

In [88]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [89]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # output_dim соответствует количеству классов

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Возвращаем тензор с размером [batch_size, output_dim]

##### Объявление функций

In [90]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")

    return result_text

In [91]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = classifier(inputs['input_ids'])
    return outputs

In [92]:
# Аугментация аудио
def augment_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise
    augmented_file_path = file_path.replace('.wav', '_augmented.wav')
    sf.write(augmented_file_path, augmented_audio, sr)
    return augmented_file_path

In [93]:
# Загрузка аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [94]:
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

In [95]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        if not mp3_filepath.endswith('.mp3'):
            print(f"Файл {mp3_filepath} не является MP3.")
            return None

        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_file(mp3_filepath)
        audio.export(wav_filepath, format='wav')
        print(f"Файл {mp3_filepath} успешно конвертирован в WAV: {wav_filepath}")
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

In [96]:
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                # Подавляем шум перед аугментацией
                augmented_file = augment_audio(audio_filepath)
                audio_files.append(augmented_file)

                text = annotation['text']
                label = annotation['label']
                texts.append(text)  # добавляем текст
                labels.append(label)  # добавляем метку

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

#### Подготовка настроек модели, создание модели

In [97]:
# Вычисление WER
def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    S = sum(1 for r, h in zip(reference_words, hypothesis_words) if r != h)
    D = len(reference_words) - len(hypothesis_words) if len(reference_words) > len(hypothesis_words) else 0
    I = len(hypothesis_words) - len(reference_words) if len(hypothesis_words) > len(reference_words) else 0
    N = len(reference_words)

    return (S + D + I) / N if N > 0 else 0

In [98]:
# Вычисление Mq
def calculate_mq(wer, f1_weighted):
    return 0.25 * (1 - wer) + 0.75 * f1_weighted

In [104]:
# Функция objective для Optuna
def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 4, 16)

    # Загрузка и подготовка данных
    audio_files, texts, labels = load_dataset()
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

    # Генерация векторов BERT
    bert_vectors = torch.stack([encode_with_bert(text) for text in train_texts])

    # Преобразование меток в тензор
    train_labels = torch.tensor(train_labels, dtype=torch.long)

    # Создание датасета и загрузчика
    train_dataset = TextDataset(bert_vectors, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Определение количества классов
    num_classes = len(set(labels))

    # Обучение модели
    classifier = TextClassifier(input_dim=bert_vectors.size(1), hidden_dim=hidden_dim, output_dim=num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    classifier.train()  # Переводим модель в режим обучения
    for epoch in range(3):  # Можно увеличить количество эпох
        for texts_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            outputs = classifier(texts_batch.float())

            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

    # Оценка модели на тестовом наборе
    classifier.eval()  # Переводим модель в режим оценки
    test_bert_vectors = torch.stack([encode_with_bert(text) for text in test_texts])
    test_dataset = TextDataset(test_bert_vectors, torch.tensor(test_labels, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for texts_batch, labels_batch in test_loader:
            outputs = classifier(texts_batch.float())
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels_batch.numpy())

    # Теперь вычисляем WER для каждого примера и усредняем
    total_wer = 0
    for ref, hyp in zip(test_texts, all_preds):
        total_wer += calculate_wer(ref, _label[hyp])  # Используем _label для преобразования предсказания в текст

    average_wer = total_wer / len(test_texts)

    # Вычисление метрик
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    mq = calculate_mq(average_wer, f1_weighted)

    return mq

In [105]:
# Настройка модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [106]:
# Оптимизация с помощью Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-10-13 02:25:01,265] A new study created in memory with name: no-name-f931751f-8435-458c-beb5-b86eaaaffa8f
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2943492142.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.


[I 2024-10-13 02:30:01,539] Trial 0 finished with value: 0.9152492088607596 and parameters: {'hidden_dim': 35, 'learning_rate': 0.003634046774341438, 'batch_size': 8}. Best is trial 0 with value: 0.9152492088607596.
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2943492142.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.


[I 2024-10-13 02:34:50,866] Trial 1 finished with value: 0.9184137658227848 and parameters: {'hidden_dim': 100, 'learning_rate': 0.0008874504786672866, 'batch_size': 6}. Best is trial 1 with value: 0.9184137658227848.
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2943492142.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...

D:\russian_railways\002_train_operator_console\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-10-13 03:04:08,776] Trial 7 finished with value: 0.8772415611814345 and parameters: {'hidden_dim': 111, 'learning_rate': 0.00039204897749052373, 'batch_size': 6}. Best is trial 2 with value: 0.9208860759493671.
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2943492142.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.


D:\russian_railways\002_train_operator_console\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-10-13 03:09:09,357] Trial 8 finished with value: 0.2122658704931697 and parameters: {'hidden_dim': 123, 'learning_rate': 3.0370658663920378e-05, 'batch_size': 16}. Best is trial 2 with value: 0.9208860759493671.
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2943492142.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.


D:\russian_railways\002_train_operator_console\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-10-13 03:13:56,844] Trial 9 finished with value: 0.1994049441949094 and parameters: {'hidden_dim': 69, 'learning_rate': 1.6518268018407632e-05, 'batch_size': 5}. Best is trial 2 with value: 0.9208860759493671.


In [107]:
print("Лучшие параметры:")
print(study.best_params)
print("Лучшее значение функции цели:")
print(study.best_value)

Лучшие параметры:
{'hidden_dim': 71, 'learning_rate': 0.0017747737172995744, 'batch_size': 6}
Лучшее значение функции цели:
0.9208860759493671


In [ ]:
# Функция предсказания команды для аудиофайла
def predict_command(audio_filepath):
    # Преобразование аудио в текст
    transcription = transcribe_audio(audio_filepath, VAL_DIR)

    # Используем обученную модель для классификации текста
    bert_vector = encode_with_bert(transcription).unsqueeze(0)  # Преобразуем вектор для одного примера
    classifier.eval()  # Убедимся, что модель находится в режиме оценки
    with torch.no_grad():
        output = classifier(bert_vector)
        _, predicted_label = torch.max(output, 1)  # Возвращаем индекс наибольшего значения (класса)

    # Возвращаем метку, которая соответствует предсказанному классу
    return _label[predicted_label.item()]

In [117]:
# Обновленная функция для оценки точности команд
def evaluate_commands():
    annotations = load_annotations(ANNOTATION_VAL_FILE)
    correct_predictions = 0
    total_predictions = len(annotations)

    # Словарь для хранения статистики по меткам
    statistics = {
        'correct': {label: 0 for label in _label.values()},
        'total': {label: 0 for label in _label.values()},
        'accuracy': {}
    }

    for item in annotations:
        audio_filepath = item['audio_filepath']
        true_label = item['label']

        audio_path = os.path.join("", audio_filepath)
        if os.path.exists(audio_path):
            # Предсказание команды
            predicted_label = predict_command(audio_path)
            print(f'Predicted: {predicted_label}, Actual: {true_label} for {audio_path}')

            # Обновление статистики
            statistics['total'][true_label] += 1
            if predicted_label == true_label:
                correct_predictions += 1
                statistics['correct'][true_label] += 1

    accuracy = correct_predictions / total_predictions

    # Сохраняем общую точность
    statistics['accuracy']['overall'] = accuracy

    # Выводим общую точность
    print(f'\nTotal Predictions: {total_predictions}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Overall Accuracy: {accuracy:.2f}')

    # Выводим статистику по каждой метке
    print('Detailed statistics per label:')
    for label in _label.values():
        total = statistics['total'][label]
        correct = statistics['correct'][label]
        label_accuracy = correct / total if total > 0 else 0
        print(f'Label: {label}, Total: {total}, Correct: {correct}, Accuracy: {label_accuracy:.2f}')


In [118]:
# Пример вызова функции
evaluate_commands()


Total Predictions: 610
Correct Predictions: 0
Overall Accuracy: 0.00
Detailed statistics per label:
Label: отказ, Total: 0, Correct: 0, Accuracy: 0.00
Label: отмена, Total: 0, Correct: 0, Accuracy: 0.00
Label: подтверждение, Total: 0, Correct: 0, Accuracy: 0.00
Label: начать осаживание, Total: 0, Correct: 0, Accuracy: 0.00
Label: осадить на (количество) вагон, Total: 0, Correct: 0, Accuracy: 0.00
Label: продолжаем осаживание, Total: 0, Correct: 0, Accuracy: 0.00
Label: зарядка тормозной магистрали, Total: 0, Correct: 0, Accuracy: 0.00
Label: вышел из межвагонного пространства, Total: 0, Correct: 0, Accuracy: 0.00
Label: продолжаем роспуск, Total: 0, Correct: 0, Accuracy: 0.00
Label: растянуть автосцепки, Total: 0, Correct: 0, Accuracy: 0.00
Label: протянуть на (количество) вагон, Total: 0, Correct: 0, Accuracy: 0.00
Label: отцепка, Total: 0, Correct: 0, Accuracy: 0.00
Label: назад на башмак, Total: 0, Correct: 0, Accuracy: 0.00
Label: захожу в межвагонное пространство, Total: 0, Corre